In [1]:
#some text pre processing tasks
#imports

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np
import pandas as pd

In [2]:
import wget
url="https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P"
path="../data"
wget.download(url,path)

100% [............................................................................] 127831 / 127831

'../data/combined_data (1).csv'

In [3]:
#reading the data

def get_data(path):
    data = pd.read_csv(path,header=0)
    return data

path="../data/combined_data.csv"

data = get_data(path)
data.head(10)

Unnamed: 0                                               text  sentiment
0           0  So there is no way for me to plug it in here i...          0
1           1                         Good case Excellent value.          1
2           2                             Great for the jawbone.          1
3           3  Tied to charger for conversations lasting more...          0
4           4                                  The mic is great.          1
5           5  I have to jiggle the plug to get it to line up...          0
6           6  If you have several dozen or several hundred c...          0
7           7        If you are Razr owner...you must have this!          1
8           8                 Needless to say I wasted my money.          0
9           9                   What a waste of money and time!.          0

In [4]:
#drop the unwanted column
data.drop(['Unnamed: 0'],axis=1)

text  sentiment
0     So there is no way for me to plug it in here i...          0
1                            Good case Excellent value.          1
2                                Great for the jawbone.          1
3     Tied to charger for conversations lasting more...          0
4                                     The mic is great.          1
...                                                 ...        ...
1987  I think food should have flavor and texture an...          0
1988                           Appetite instantly gone.          0
1989  Overall I was not impressed and would not go b...          0
1990  The whole experience was underwhelming and I t...          0
1991  Then as if I hadn't wasted enough of my life t...          0

[1992 rows x 2 columns]

In [5]:
# clone package repository
!git clone https://github.com/vallantin/atalaia.git

fatal: destination path 'atalaia' already exists and is not an empty directory.


In [6]:
# install packages requirements
!pip install -r requirements.txt

# install package
!python setup.py install

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
(null): can't open file 'setup.py': [Errno 2] No such file or directory


In [7]:
from atalaia.atalaia import Atalaia

#preprocess function from atalaia module
def preprocess(panda_series):
    atalaia = Atalaia('en')

    # lower case everyting and remove double spaces
    panda_series = (atalaia.lower_remove_white(t) for t in panda_series)

    # expand contractions
    panda_series = (atalaia.expand_contractions(t) for t in panda_series)

    # remove punctuation
    panda_series = (atalaia.remove_punctuation(t) for t in panda_series)

    # remove numbers
    panda_series = (atalaia.remove_numbers(t) for t in panda_series)

    # remove stopwords
    panda_series = (atalaia.remove_stopwords(t) for t in panda_series)

    # remove excessive spaces
    panda_series = (atalaia.remove_excessive_spaces(t) for t in panda_series)

    return panda_series

# preprocess it
preprocessed_text = preprocess(data.text)

# assign preprocessed texts to dataset
data['text'] = list(preprocessed_text)

# see data
data.head(5)

Unnamed: 0                                               text  sentiment
0           0                 no way plug us unless go converter          0
1           1                          good case excellent value          1
2           2                                      great jawbone          1
3           3  tied charger conversations lasting minutes maj...          0
4           4                                          mic great          1

In [8]:
# shuffle the dataset
data = data.sample(frac=1)

# separate all classes present on the dataset
classes_dict = {}
for label in [0,1]:
    classes_dict[label] = data[data['sentiment'] == label]

# get 80% of each label
size = int(len(classes_dict[0].text) * 0.8)
X_train = list(classes_dict[0].text[0:size])      + list(classes_dict[1].text[0:size])
X_test  = list(classes_dict[0].text[size:])       + list(classes_dict[1].text[size:])
y_train = list(classes_dict[0].sentiment[0:size]) + list(classes_dict[1].sentiment[0:size])
y_test  = list(classes_dict[0].sentiment[size:])  + list(classes_dict[1].sentiment[size:])

In [9]:
# Convert labels to Numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

In [10]:
# Let's consider the vocab size as the number of words
# that compose 90% of the vocabulary
atalaia    = Atalaia('en')
vocab_size = len(atalaia.representative_tokens(0.9, 
                                               ' '.join(X_train),
                                               reverse=False))
oov_tok = "<OOV>"

# start tokenize
tokenizer = Tokenizer(num_words=vocab_size, 
                      oov_token=oov_tok)

# fit on training
# we don't fit on test because, in real life, our model will have to deal with
# words ir never saw before. So, it makes sense fitting only on training.
# when it finds a word it never saw before, it will assign the 
# <OOV> tag to it.
tokenizer.fit_on_texts(X_train)

# get the word index
word_index = tokenizer.word_index

# transform into sequences
# this will assign a index to the tokens present on the corpus
sequences = tokenizer.texts_to_sequences(X_train)

# see the first sequence
sequences[0]

[1058, 125, 58, 168, 2, 169]

In [11]:
# define max_length 
max_length = 100

# post: pad or truncate after sentence.
# pre: pad or truncate before sentence.
trunc_type='post'
padding_type='post'

padded = pad_sequences(sequences,
                       maxlen=max_length, 
                       padding=padding_type, 
                       truncating=trunc_type)

# tokenize and pad test sentences
# thse will be used later on the model for accuracy test
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_test_padded    = pad_sequences(X_test_sequences,
                                 maxlen=max_length, 
                                 padding=padding_type, 
                                 truncating=trunc_type)

# check the first padded sentence. Notice that 0s were added to it
# because it was shorter than 100
padded[0]

array([1058,  125,   58,  168,    2,  169,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])

In [12]:
# create the reverse word index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# create the decoder
def text_decoder(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

# print the decoder output for one sentence and compare it to original
print('Decoded sentence:')
print(text_decoder(padded[1]))
print('\nOriginal sentence')
print(X_train[1])

Decoded sentence:
say bye bye tip lady ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?

Original sentence
say bye bye tip lady


In [13]:
# Build network
embedding_dim = 16

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           28512     
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 9606      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 38,125
Trainable params: 38,125
Non-trainable params: 0
_________________________________________________________________


In [14]:
num_epochs = 10
model.fit(padded, y_train, epochs=num_epochs, validation_data=(X_test_padded, y_test))

Epoch 1/10
50/50 [==============================] - 1s 12ms/step - loss: 0.6931 - accuracy: 0.5314 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 2/10
50/50 [==============================] - 0s 6ms/step - loss: 0.6894 - accuracy: 0.6206 - val_loss: 0.6894 - val_accuracy: 0.5325
Epoch 3/10
50/50 [==============================] - 0s 5ms/step - loss: 0.6510 - accuracy: 0.8109 - val_loss: 0.6671 - val_accuracy: 0.6350
Epoch 4/10
50/50 [==============================] - 0s 5ms/step - loss: 0.5220 - accuracy: 0.9070 - val_loss: 0.6842 - val_accuracy: 0.6525
Epoch 5/10
50/50 [==============================] - 0s 5ms/step - loss: 0.3721 - accuracy: 0.9108 - val_loss: 0.8384 - val_accuracy: 0.6400
Epoch 6/10
50/50 [==============================] - 0s 5ms/step - loss: 0.2843 - accuracy: 0.9215 - val_loss: 0.7647 - val_accuracy: 0.6800
Epoch 7/10
50/50 [==============================] - 0s 4ms/step - loss: 0.2307 - accuracy: 0.9410 - val_loss: 0.8413 - val_accuracy: 0.6850
Epoch 8/10
50/50 [=

In [15]:
test_loss, test_acc = model.evaluate(X_test_padded, y_test, verbose=2)
print('\nModel accuracy: {:.0f}%'.format(test_acc*100))

13/13 - 0s - loss: 0.7710 - accuracy: 0.7350

Model accuracy: 74%


In [16]:
# Use the model to predict new reviews   
new_reviews = ['Nothing could smell better than this fragrance.', 'Everything was perfect','They respect the environment.', 'The cake was a little dry','Everything was terrible.','it didn\'t work as expected']

# preprocess the texts
new_reviews = list(preprocess(new_reviews))
print(new_reviews)

['nothing smell better fragrance', 'everything perfect', 'respect environment', 'cake little dry', 'everything terrible', 'not work expected']


In [17]:
# Create the sequences
padding_type     = 'post'
new_sequences    = tokenizer.texts_to_sequences(new_reviews)
new_padded       = pad_sequences(new_sequences, padding=padding_type, maxlen=max_length)           

# predict
y_pred           = model.predict(X_test_padded)

In [18]:
# See the predictions
for x in range(len(new_reviews)):
  print(new_reviews[x])
  print(y_pred[x])
  print('\n')

nothing smell better fragrance
[0.88830435]


everything perfect
[0.32505876]


respect environment
[0.46855438]


cake little dry
[0.10603553]


everything terrible
[0.9970948]


not work expected
[0.04201415]




In [19]:
# round
y_pred = [1 if y > 0.5 else 0 for y in y_pred]

In [20]:
# confusion matrix
matrix = tf.math.confusion_matrix(y_test, y_pred)

matrix = np.array(matrix)

matrix = pd.DataFrame(matrix, columns=['Positive (real)', 'Negative (real)'], index=['Positive (predicted)', 'Negative (predicted)'])

In [21]:
matrix

Positive (real)  Negative (real)
Positive (predicted)              107               93
Negative (predicted)               13              187